In [31]:
import pandas, geopandas, numpy, os

GGH_Census_06 = pandas.read_csv("../census-summary-data-2006-2021/GGH-Census-da-06.csv")
GGH_Census_21 = pandas.read_csv("../census-summary-data-2006-2021/GGH-Census-da-21.csv")

folder = "D:/Softwares/OneDrive - Amdev Property/3 School of Cities/Census Data Analysis/Data/Analysis/IntersectedDA"

transit_buffer_06 = geopandas.read_file (os.path.join(folder,"tbf_DA_06_Perc80OR1000.shp"))
transit_buffer_21 = geopandas.read_file (os.path.join(folder,"tbf_DA_21_Perc80OR1000.shp"))
ugc_06 = geopandas.read_file (os.path.join(folder,"ugc_DA_06_Perc80OR1000.shp"))
ugc_21 = geopandas.read_file (os.path.join(folder,"ugc_DA_21_Perc80OR1000.shp"))
built_bound_06 = geopandas.read_file (os.path.join(folder,"blt_DA_06_Perc80OR1000.shp"))
built_bound_21 = geopandas.read_file (os.path.join(folder,"blt_DA_21_Perc80OR1000.shp"))
GGH_Census_06 = GGH_Census_06.drop("Unnamed: 0", axis =1)
GGH_Census_21 = GGH_Census_21.drop("Unnamed: 0", axis =1)

In [32]:
transit_buffer_06_DAUID = transit_buffer_06["DAUID"].copy().reset_index().drop('index', axis =1)
transit_buffer_21_DAUID = transit_buffer_21["DAUID"].copy().reset_index().drop('index', axis =1)
ugc_06_DAUID = ugc_06["DAUID"].copy().reset_index().drop('index', axis =1)
ugc_21_DAUID = ugc_21["DAUID"].copy().reset_index().drop('index', axis =1)
built_bound_06_DAUID = built_bound_06["DAUID"].copy().reset_index().drop('index', axis =1)
built_bound_21_DAUID = built_bound_21["DAUID"].copy().reset_index().drop('index', axis =1)
ugc_06_DAUID.to_csv("../census-file-handling/Data/ugc.csv")


In [33]:
#set field type to string to allow merging tables. 
ugc_06_DAUID['DAUID'] = ugc_06_DAUID['DAUID'].astype('str')
ugc_21_DAUID['DAUID'] = ugc_21_DAUID['DAUID'].astype('str')
built_bound_06_DAUID['DAUID'] = built_bound_06_DAUID['DAUID'].astype('str')
built_bound_21_DAUID['DAUID'] = built_bound_21_DAUID['DAUID'].astype('str')
transit_buffer_06_DAUID['DAUID'] = transit_buffer_06_DAUID['DAUID'].astype('str')
transit_buffer_21_DAUID['DAUID'] = transit_buffer_21_DAUID['DAUID'].astype('str')
GGH_Census_06['DAUID'] = GGH_Census_06['DAUID'].astype('str')
GGH_Census_21['DAUID'] = GGH_Census_21['DAUID'].astype('str')

#merge the selected DAs with census data based on DAUID. 
ugc_06 = pandas.merge(ugc_06_DAUID, GGH_Census_06, on = "DAUID")
ugc_21 = pandas.merge(ugc_21_DAUID, GGH_Census_21, on = "DAUID")
built_bound_06 = pandas.merge(built_bound_06_DAUID, GGH_Census_06, on = "DAUID")
built_bound_21 = pandas.merge(built_bound_21_DAUID, GGH_Census_21, on = "DAUID")
transit_buffer_06 = pandas.merge(transit_buffer_06_DAUID, GGH_Census_06, on = "DAUID")
transit_buffer_21 = pandas.merge(transit_buffer_21_DAUID, GGH_Census_21, on = "DAUID")

# Pivot table to dissolve the records by Municipality
ugc_06 = pandas.pivot_table(data = ugc_06, values = ugc_06.columns.tolist()[4:], index = ['Municipality'], aggfunc = numpy.sum)
ugc_21 = pandas.pivot_table(data = ugc_21, values = ugc_21.columns.tolist()[4:], index = ['Municipality'], aggfunc = numpy.sum)
built_bound_06 = pandas.pivot_table(data = built_bound_06, values = built_bound_06.columns.tolist()[4:], index = ['Municipality'], aggfunc = numpy.sum)
built_bound_21 = pandas.pivot_table(data = built_bound_21, values = built_bound_21.columns.tolist()[4:], index = ['Municipality'], aggfunc = numpy.sum)
transit_buffer_06 = pandas.pivot_table(data = transit_buffer_06, values = transit_buffer_06.columns.tolist()[4:], index = ['Municipality'], aggfunc = numpy.sum)
transit_buffer_21 = pandas.pivot_table(data = transit_buffer_21, values = transit_buffer_21.columns.tolist()[4:], index = ['Municipality'], aggfunc = numpy.sum)

# join tables
ugc = pandas.merge(ugc_06, ugc_21, on = 'Municipality').reset_index()
built_bound = pandas.merge(built_bound_06, built_bound_21, on = 'Municipality').reset_index()
transit_buffer = pandas.merge(transit_buffer_06, transit_buffer_21, on = 'Municipality').reset_index()


In [40]:
def changeField(inTable, field_indentifier):
    pivot_field = inTable.columns.tolist()
    i = 1
    #calculate the change between 2006 and 2021
    while i <= (len(pivot_field)-1)/2:
        #feed 2006's census field name for the "change" section
        changeField = pivot_field[i].replace("2006", "Chg") 
        #create the change field by subtracting 2006 information from 2021 information. 
        inTable[f'{changeField}'] = (inTable [pivot_field[i+12]] - inTable [pivot_field[i]]) / inTable [pivot_field[i]]
        i+=1

    #rename the column, remove the alphabets added to keep the columns in place. 
    i=1
    column = inTable.columns.tolist()
    while i < len(column):
        columnList = column[i].split("-")
        print(columnList)
        inTable = inTable.rename(columns={f"{column[i]}": f"{columnList[0]} {field_indentifier} {columnList[2]}"})

        print({f"{column[i]}": f"{columnList[0]} {columnList[2]}"})
        i+=1
    return inTable
    
ugc_summary = changeField(ugc, "ugc")
built_boundary_summary = changeField(built_bound, "blt")
transit_buffer_summary = changeField(transit_buffer, "tbf")

['2006', 'a', 'Population']
{'2006-a-Population': '2006 Population'}
['2006', 'b', 'Total private dwellings']
{'2006-b-Total private dwellings': '2006 Total private dwellings'}
['2006', 'c', 'Total dwellings structures']
{'2006-c-Total dwellings structures': '2006 Total dwellings structures'}
['2006', 'd', 'Single', 'detached house']
{'2006-d-Single-detached house': '2006 Single'}
['2006', 'e', 'Semi', 'detached house']
{'2006-e-Semi-detached house': '2006 Semi'}
['2006', 'f', 'Row house']
{'2006-f-Row house': '2006 Row house'}
['2006', 'g', 'Apartment, duplex']
{'2006-g-Apartment, duplex': '2006 Apartment, duplex'}
['2006', 'h', 'Apartment, building that has five or more storeys']
{'2006-h-Apartment, building that has five or more storeys': '2006 Apartment, building that has five or more storeys'}
['2006', 'i', 'Apartment, building that has fewer than five storeys']
{'2006-i-Apartment, building that has fewer than five storeys': '2006 Apartment, building that has fewer than five store

In [42]:
a = built_boundary_summary.columns.tolist()
i = 0
while i < len(a):
    print(i, a[i])
    i +=1


0 Municipality
1 2006 blt Population
2 2006 blt Total private dwellings
3 2006 blt Total dwellings structures
4 2006 blt Single
5 2006 blt Semi
6 2006 blt Row house
7 2006 blt Apartment, duplex
8 2006 blt Apartment, building that has five or more storeys
9 2006 blt Apartment, building that has fewer than five storeys
10 2006 blt Other single
11 2006 blt Movable dwelling
12 2006 blt Total households
13 2021 blt Population
14 2021 blt Total private dwellings
15 2021 blt Total dwelling type
16 2021 blt Single
17 2021 blt Semi
18 2021 blt Row house
19 2021 blt Apartment or flat in a duplex
20 2021 blt Apartment in a building that has five or more storeys
21 2021 blt Apartment in a building that has fewer than five storeys
22 2021 blt Other single
23 2021 blt Movable dwelling
24 2021 blt Total households
25 Chg blt Population
26 Chg blt Total private dwellings
27 Chg blt Total dwellings structures
28 Chg blt Single
29 Chg blt Semi
30 Chg blt Row house
31 Chg blt Apartment, duplex
32 Chg blt

In [29]:
ugc.to_csv("../census-file-handling/Data/ugc.csv")